<a href="https://colab.research.google.com/github/MD2001/Pytorch_Bignnier/blob/main/01_pytoirch_WorkFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Title: 00. PyTorch Fundamentals
Resource notebook: https://www.learnpytorch.io/00_pytorch_fundamentals/

Discussion forum: https://github.com/mrdbourke/pytorch-deep-learning/discussions

In [108]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print(torch.__version__)

2.8.0+cu126


## introduction
### tensors

In [109]:
#scaler
scaler=torch.tensor(7)
scaler

tensor(7)

In [110]:
scaler.shape

torch.Size([])

In [111]:
scaler.ndim

0

In [112]:
#Vector
Vector=torch.tensor([7,7])
Vector

tensor([7, 7])

In [113]:
Vector.shape

torch.Size([2])

In [114]:
Vector.ndim

1

In [115]:
#MATRIX
MATRIX =torch.tensor([[1,3],[2,4]])
MATRIX

tensor([[1, 3],
        [2, 4]])

In [116]:
MATRIX.shape

torch.Size([2, 2])

In [117]:
MATRIX.ndim

2

In [118]:
MATRIX[0]

tensor([1, 3])

In [119]:
MATRIX[1]

tensor([2, 4])

In [120]:
#Tensors
TENSOR = torch.tensor([[[1,2,3,4],[5,6,7,8],[9,10,11,12]]])
TENSOR

tensor([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]]])

In [121]:
TENSOR.shape

torch.Size([1, 3, 4])

In [122]:
TENSOR.ndim

3

In [123]:
TENSOR[0]

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])

In [124]:
Tesnor2=torch.tensor([[[1,2],[3,4],[5,6]],[[7,8],[9,10],[11,12]]])
Tesnor2

tensor([[[ 1,  2],
         [ 3,  4],
         [ 5,  6]],

        [[ 7,  8],
         [ 9, 10],
         [11, 12]]])

In [125]:
Tesnor2.ndim

3

In [126]:
Tesnor2.shape

torch.Size([2, 3, 2])

## Random tensors
### Why random tensors?
### Random tensors are important because the way many neural networks learn is that they start with tensors full of random numbers and then
### Torch random tensors-https://docs.pytorch.org/docs/stable/generated/torch.rand.html

In [127]:
tensore_rand=torch.rand(3,4)
tensore_rand

tensor([[0.4216, 0.2373, 0.8111, 0.5340],
        [0.6834, 0.6528, 0.6123, 0.9834],
        [0.1826, 0.1239, 0.4262, 0.9717]])

In [128]:
tensore_rand2=torch.rand(size=(3,224,224))
tensore_rand2

tensor([[[0.6248, 0.7822, 0.4098,  ..., 0.5574, 0.4617, 0.5910],
         [0.8125, 0.1991, 0.8147,  ..., 0.0305, 0.8391, 0.8155],
         [0.1815, 0.1551, 0.4583,  ..., 0.9515, 0.5850, 0.0560],
         ...,
         [0.9278, 0.8557, 0.6127,  ..., 0.6272, 0.7446, 0.9792],
         [0.8955, 0.5506, 0.2410,  ..., 0.5146, 0.7194, 0.1651],
         [0.6431, 0.3780, 0.2382,  ..., 0.2936, 0.6081, 0.0819]],

        [[0.6269, 0.4986, 0.6736,  ..., 0.5330, 0.8418, 0.1268],
         [0.6790, 0.5004, 0.4073,  ..., 0.4173, 0.5488, 0.2828],
         [0.0696, 0.9972, 0.0032,  ..., 0.9242, 0.1857, 0.2759],
         ...,
         [0.2403, 0.4134, 0.4760,  ..., 0.7852, 0.8114, 0.7579],
         [0.1290, 0.5669, 0.3054,  ..., 0.6924, 0.5945, 0.9664],
         [0.8446, 0.2196, 0.9623,  ..., 0.2774, 0.7709, 0.5955]],

        [[0.2027, 0.8164, 0.5923,  ..., 0.6365, 0.7605, 0.8878],
         [0.0560, 0.6202, 0.1344,  ..., 0.7107, 0.2632, 0.7326],
         [0.7437, 0.8904, 0.8845,  ..., 0.5876, 0.1500, 0.

In [129]:
tensore_rand2.ndim , tensore_rand2.shape

(3, torch.Size([3, 224, 224]))

## zeros or ones arrays useed often to mask biger arrays

In [130]:
zeros=torch.zeros(size=(3,4))
zeros

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [131]:
ones=torch.ones(size=(3,4))
ones

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [132]:
aranges=torch.arange(1,11,1)
aranges

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [133]:
zeros_like=torch.zeros_like(input=aranges)
zeros_like

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Tensor datatypes
**Note:** Tensor datatypes is one of the 3 big errors you'll run into with PyTorch
& deep learning:
1. Tensors not right datatype
2. Tensors not right shape
3. Tensors not on the right device

In [134]:
float_32_tensor=torch.tensor([3.0,6.0,9.0],
                             dtype=None,
                             device=None,
                             requires_grad=False)
float_32_tensor

tensor([3., 6., 9.])

In [135]:
float_16_tensor=float_32_tensor.type(torch.float16)
float_16_tensor

tensor([3., 6., 9.], dtype=torch.float16)

In [136]:
float_16_tensor*float_32_tensor ## why this work?

tensor([ 9., 36., 81.])

In [137]:
int_32_tensor=torch.tensor([1,2,3],dtype=torch.int32)
int_32_tensor

tensor([1, 2, 3], dtype=torch.int32)

In [138]:
float_16_tensor*int_32_tensor

tensor([ 3., 12., 27.], dtype=torch.float16)

In [139]:
some_tensor=torch.rand(3,4)
some_tensor

tensor([[0.8374, 0.9304, 0.4142, 0.7294],
        [0.2263, 0.4782, 0.5584, 0.6144],
        [0.6632, 0.9714, 0.2523, 0.9516]])

In [140]:
print(some_tensor)
print(f"this is the dtype of tensor: {some_tensor.dtype}")
print(f"this is the shape of tenosr: {some_tensor.shape}")
print(f"this is the device of tensor: {some_tensor.device}")

tensor([[0.8374, 0.9304, 0.4142, 0.7294],
        [0.2263, 0.4782, 0.5584, 0.6144],
        [0.6632, 0.9714, 0.2523, 0.9516]])
this is the dtype of tensor: torch.float32
this is the shape of tenosr: torch.Size([3, 4])
this is the device of tensor: cpu


##math in tensors

In [141]:
tensor=torch.tensor([1,2,3])
tensor

tensor([1, 2, 3])

In [142]:
tensor+10

tensor([11, 12, 13])

In [143]:
tensor*10

tensor([10, 20, 30])

In [144]:
tensor-10

tensor([-9, -8, -7])

In [145]:
torch.add(tensor,10)

tensor([11, 12, 13])

Matrix multiplication
Two main ways of performing multiplication in neural networks and deep learning:
1. Element-wise multiplication
2. Matrix mutliplication (dot product)
More information on multiplying matrices -  https://www.mathsisfun.com/algebra/matrix-multiplying.html


In [146]:
%%time
torch.matmul(tensor,tensor)
## since the total time is 10 time or more less than for loop this method is too better

CPU times: user 403 µs, sys: 29 µs, total: 432 µs
Wall time: 1.69 ms


tensor(14)

In [147]:
%%time
value = 0
for i in range(len(tensor)):
  value += tensor[i] * tensor[i]
print(value)

tensor(14)
CPU times: user 889 µs, sys: 66 µs, total: 955 µs
Wall time: 1.68 ms


* short cut for torch.matmul is @
1. the first rule in matrxt multiblication is the inner dimintion should be equla
- for example:
* '(3,2)'@'(3,2)' not work
* '(3,2)'@'(2,3)' will work
* '(2,3)'@'(3,2)' will work
2. the resultin is the outer dimintion

In [148]:
(torch.rand(3,2) @ torch.rand(2,3) ).shape

torch.Size([3, 3])

#the transpose is importatint how to use it

In [149]:
tensor=torch.rand(3,2)
tensor

tensor([[0.7803, 0.4587],
        [0.5491, 0.4142],
        [0.5111, 0.1247]])

In [150]:
tensor @ tensor.T

tensor([[0.8192, 0.6184, 0.4559],
        [0.6184, 0.4730, 0.3323],
        [0.4559, 0.3323, 0.2767]])

In [151]:
x=torch.arange(0,100,10)
x

tensor([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])

In [152]:
torch.max(x),x.max()

(tensor(90), tensor(90))

In [153]:
torch.min(x),x.min()

(tensor(0), tensor(0))

In [154]:
torch.mean(x.type(torch.float32)),x.type(torch.float32).mean()

(tensor(45.), tensor(45.))

In [155]:
torch.sum(x),x.sum()

(tensor(450), tensor(450))

In [156]:
x.argmax()

tensor(9)

In [157]:
x.argmin()

tensor(0)

##Reshaping, stacking, squeezing and unsqueezing tensors
* Reshaping - reshapes an input tensor to a defined shape
View
* Return a view of an input tensor of certain shape but keep the same memory as the original tensor
Stacking
* combine multiple tensors on top of each other (vstack) or side by side (hstack)

* Squeeze removes all 1 dimensions from a tensor

* Unsqueeze add a 1 dimension to a target tensor

* Permute Return a view of the input with dimensions permuted (swapped) in a certain way

In [158]:
tensor=torch.arange(1,10)
tensor,tensor.shape

(tensor([1, 2, 3, 4, 5, 6, 7, 8, 9]), torch.Size([9]))

In [159]:
tensor_reshape=tensor.reshape(1,9)
tensor_reshape,tensor_reshape.shape

(tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9]]), torch.Size([1, 9]))

In [160]:
tensor_reshape=tensor.reshape(3,3)
tensor_reshape,tensor_reshape.shape

(tensor([[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]),
 torch.Size([3, 3]))

In [161]:
tensor , tensor_reshape

(tensor([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 tensor([[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]))

In [162]:
tensor[0]=66
tensor , tensor_reshape

(tensor([66,  2,  3,  4,  5,  6,  7,  8,  9]),
 tensor([[66,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9]]))

In [163]:
x=torch.arange(1,10)
x ,x.shape

(tensor([1, 2, 3, 4, 5, 6, 7, 8, 9]), torch.Size([9]))

In [164]:
z=x.view(3,3)
z

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [165]:
x[0]=55
x,z
#reshape and view do the same

(tensor([55,  2,  3,  4,  5,  6,  7,  8,  9]),
 tensor([[55,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9]]))

**contiguous memore** : "each row follows the previous one directly in memory"
* view : contiguous it work fine but if not not work
* reshape : if not contiguous it rearnge it and work fine.

**then what make tensor not contiguous ??**

In [166]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([4, 5, 6])
x = torch.stack((a, b))
x,x.shape

(tensor([[1, 2, 3],
         [4, 5, 6]]),
 torch.Size([2, 3]))

the .stack() functions add two tensore together
* but using dim attribute behave like: "where the index of new diminshin will be"

  and this shape how the output will be

In [167]:
x = torch.stack((a, b),dim=0)
x,x.shape

(tensor([[1, 2, 3],
         [4, 5, 6]]),
 torch.Size([2, 3]))

In [168]:
x = torch.stack((a, b),dim=1)
x,x.shape

(tensor([[1, 4],
         [2, 5],
         [3, 6]]),
 torch.Size([3, 2]))

### try higher diminshions

In [169]:
a = torch.randn(2, 3)
b = torch.randn(2, 3)
d = torch.randn(2, 3)
print(f"The stack when dim is 0 :\n{torch.stack((a, b, d), dim=0)}")
print(f"The shpae of stack when The dim is 0 :\n{torch.stack((a, b, d), dim=0).shape}")
print("************************************************************************************************")
print(f"The stack when dim is 1 :\n{torch.stack((a, b, d), dim=1)}")
print(f"The shpae of stack when The dim is 1 :\n{torch.stack((a, b, d), dim=1).shape}")
print("************************************************************************************************")
print(f"The stack when dim is 2 :\n{torch.stack((a, b, d), dim=2)}")
print(f"The shpae of stack when The dim is 2 :\n{torch.stack((a, b, d), dim=2).shape}")
## the dim = 3 cuse error

The stack when dim is 0 :
tensor([[[-0.4231, -0.2759, -0.1044],
         [ 2.4374,  0.9525, -1.6060]],

        [[ 0.5182,  1.1752,  0.0723],
         [ 1.5628,  0.2253,  0.4295]],

        [[ 0.3529,  1.6247, -2.3077],
         [-0.7570,  1.6156,  0.2700]]])
The shpae of stack when The dim is 0 :
torch.Size([3, 2, 3])
************************************************************************************************
The stack when dim is 1 :
tensor([[[-0.4231, -0.2759, -0.1044],
         [ 0.5182,  1.1752,  0.0723],
         [ 0.3529,  1.6247, -2.3077]],

        [[ 2.4374,  0.9525, -1.6060],
         [ 1.5628,  0.2253,  0.4295],
         [-0.7570,  1.6156,  0.2700]]])
The shpae of stack when The dim is 1 :
torch.Size([2, 3, 3])
************************************************************************************************
The stack when dim is 2 :
tensor([[[-0.4231,  0.5182,  0.3529],
         [-0.2759,  1.1752,  1.6247],
         [-0.1044,  0.0723, -2.3077]],

        [[ 2.4374,  1.56

###the squeeze tensors

In [170]:
tensor=torch.arange(1,10).reshape(1,9)
tensor, tensor.shape

print(f"the tensore before squezze:{tensor}")
print(f"the shape before squezze:{tensor.shape}")
print("************************************************************************************************")
print(f"the tensore after squezze:{tensor.squeeze()}")
print(f"the shape after squezze:{tensor.squeeze().shape}")

the tensore before squezze:tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9]])
the shape before squezze:torch.Size([1, 9])
************************************************************************************************
the tensore after squezze:tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])
the shape after squezze:torch.Size([9])


In [171]:
tensor=torch.arange(1,10)
tensor, tensor.shape

print(f"the tensore before unsquezze:{tensor}")
print(f"the shape before unsquezze:{tensor.shape}")
print("************************************************************************************************")
print(f"the tensore after unsquezze dim = 0:{tensor.unsqueeze(dim=0)}")
print(f"the shape after unsquezze dim = 0:{tensor.unsqueeze(dim=0).shape}")
print("************************************************************************************************")
print(f"the tensore after unsquezze dim = 1:{tensor.unsqueeze(dim=1)}")
print(f"the shape after unsquezze dim = 1:{tensor.unsqueeze(dim=1).shape}")

the tensore before unsquezze:tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])
the shape before unsquezze:torch.Size([9])
************************************************************************************************
the tensore after unsquezze dim = 0:tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9]])
the shape after unsquezze dim = 0:torch.Size([1, 9])
************************************************************************************************
the tensore after unsquezze dim = 1:tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]])
the shape after unsquezze dim = 1:torch.Size([9, 1])


In [172]:
orginal_tensor=torch.rand(size=(5,6,7))
tranc_tensor=orginal_tensor.permute(2,0,1)
print(f"orginal_tensor shape:{orginal_tensor.shape}")
print(f"trans_tensor shape:{tranc_tensor.shape}")
print(f"orginal_tensor shape:{orginal_tensor.shape}")
## poth share the same memory but have diffrent view
print(f"the index on orginal tensor:{orginal_tensor[0,0,0]}")
print(f"the index on trans_tensor:{tranc_tensor[0,0,0]}")
orginal_tensor[0,0,0]=2
print(f"the index on orginal tensor after change:{orginal_tensor[0,0,0]}")
print(f"the index on trans_tensor after change:{tranc_tensor[0,0,0]}")



orginal_tensor shape:torch.Size([5, 6, 7])
trans_tensor shape:torch.Size([7, 5, 6])
orginal_tensor shape:torch.Size([5, 6, 7])
the index on orginal tensor:0.3541228771209717
the index on trans_tensor:0.3541228771209717
the index on orginal tensor after change:2.0
the index on trans_tensor after change:2.0


# indexing

In [173]:
x=torch.arange(1,10).reshape(1,3,3)
x,x.shape

(tensor([[[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]]]),
 torch.Size([1, 3, 3]))

In [174]:
x[0]

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [175]:
x[0][0]

tensor([1, 2, 3])

In [176]:
#seperte [n][n][n] is equal to [n,n,n]
x[0,0,0]

tensor(1)

In [177]:
# we use : to say select all elemnt in this diminshine
x[:,0]

tensor([[1, 2, 3]])

In [178]:
new=torch.arange(11,20).reshape(1,3,3)
x=torch.cat((x,new),dim=0)
x

tensor([[[ 1,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9]],

        [[11, 12, 13],
         [14, 15, 16],
         [17, 18, 19]]])

In [179]:
x[:,0]

tensor([[ 1,  2,  3],
        [11, 12, 13]])

In [180]:
x[:,0,2]

tensor([ 3, 13])

## get start from numpy to tensor

In [181]:
import numpy as np
array=np.arange(1,8)
tensor=torch.from_numpy(array)
array,tensor

(array([1, 2, 3, 4, 5, 6, 7]), tensor([1, 2, 3, 4, 5, 6, 7]))

In [182]:
## every time you run this line you get differant tensor
rand_tensor=torch.rand(3,4)
rand_tensor


tensor([[0.0539, 0.1522, 0.6414, 0.8040],
        [0.9498, 0.8600, 0.5898, 0.7526],
        [0.3401, 0.6602, 0.1139, 0.4753]])

In [183]:
## always get the same set of number
torch.manual_seed(42)
rand_tensor=torch.rand(3,4)
rand_tensor

tensor([[0.8823, 0.9150, 0.3829, 0.9593],
        [0.3904, 0.6009, 0.2566, 0.7936],
        [0.9408, 0.1332, 0.9346, 0.5936]])

In [184]:
!nvidia-smi ## do it after reasigne the runtime hardware to gpu

Tue Oct 28 12:05:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   76C    P0             32W /   70W |     120MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [185]:
device= "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [186]:
torch.cuda.is_available()

True

In [187]:
torch.cuda.device_count()

1

In [188]:
torch.manual_seed(45) ## this function her because no reasone just i try it
tensor=torch.rand(3,4)
print(tensor,tensor.device)

tensor([[0.1869, 0.9613, 0.6834, 0.8988],
        [0.0505, 0.5555, 0.7861, 0.0566],
        [0.7842, 0.1480, 0.0388, 0.1037]]) cpu


In [189]:
tensor=tensor.to(device)
print(tensor ,tensor.device)


tensor([[0.1869, 0.9613, 0.6834, 0.8988],
        [0.0505, 0.5555, 0.7861, 0.0566],
        [0.7842, 0.1480, 0.0388, 0.1037]], device='cuda:0') cuda:0


In [190]:
## Note: if the tensore on the gpu can't move to numpy ,,, to do it you have get back to cpu then you can move tensor to numpy
tensor_back_to_cpu=tensor.cpu().numpy()
tensor_back_to_cpu

array([[0.18694752, 0.9613284 , 0.683449  , 0.89879584],
       [0.0505085 , 0.5554561 , 0.7861288 , 0.0566228 ],
       [0.7841894 , 0.14798379, 0.03884083, 0.1037364 ]], dtype=float32)

## Exirsies